In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
%run supported_models/sklRF
%run dataset
%run z3

In [22]:
ds = download_openml_dataset( 'vote' )
clf = RandomForestClassifier()
clf = clf.fit(ds.data, ds.target)
RF = SciKitLearnRF(clf,ds)
engine = Z3(ds,RF)

/zfsauton/home/ngisolfi/anaconda2/envs/p3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [23]:
for rowID in range(5):
    query={}
    for attnum in range(len(ds.data.columns)):
        query[ds.data.columns[attnum]]=ds.data.iloc[rowID][attnum]

    scriptText = \
        engine.set_options_true(['produce-models',
                                 'produce-unsat-assumptions',
                                 'produce-unsat-cores',
                                 'smt.core.minimize']) + \
        engine.set_logic('AUFLIRA') + \
        engine.declare_query_attribute_values() + \
        engine.assert_query_attribute_values(query)+ \
        engine.declare_query_target() + \
        engine.assert_query_target(ds.target[rowID]) + \
        engine.declare_ensemble_prediction_array() + \
        engine.declare_thresholds() + \
        engine.assert_thresholds() + \
        engine.declare_rules() + \
        engine.assert_rules() + \
        engine.declare_branches() + \
        engine.assert_branches() + \
        engine.declare_predictions() + \
        engine.assert_leaves() + \
        engine.assert_ensemble_prediction_array()

    for i in range(len(engine.dataSet.targetMap)):
        scriptText += engine.declare_function_count_key_in_array(i)
        scriptText += engine.assert_function_count_key_in_array(i)

    scriptText += \
        engine.declare_ensemble_prediction_votes() + \
        engine.assert_ensemble_prediction_votes() + \
        engine.declare_function_max_of_array() + \
        engine.assert_function_max_of_array() + \
        engine.declare_human_expectation_for_query_target() + \
        engine.assert_human_expectation_for_query_target(1-ds.target[rowID]) + \
        engine.conclude()

    filename = 'output/why_not_row%d_prediction'%(rowID)
    script2file(filename,scriptText)
    run_z3_script( filename, '.model' )
